### Final year project - Multipliers for difference sets
Samuel Hudson and Lijun Zou<br/>
NUI Galway

## 1. Parameters of difference sets.
Let $G$ be a finite group of order $v$. A non-empty subset $D\subset G$ of size $k<v$ is a $(v,k,\lambda)$-**difference set** if each element of $G\setminus \{1_G\}$ occurs exactly $\lambda$ times in the the multiset 
    $$
    \Delta(D) = \{
    d_1{d_2}^{-1}\mid 
    d_1,d_2 \in D, d_1 \neq d_2 \}.$$

We first write a function checking difference sets in cyclic groups.

In [99]:
import math,time,itertools  #import the libraries we are going to use
from math import sqrt

In [88]:
#difference_set(l,n) returns True if elements in the list l form a difference set in Z_n
def difference_set(l,n):
    delta = [] #multiset of differences
    for i in l:
        for j in l:
            delta.append((i-j)%n)
    l = delta.count(1)        
    for i in range(2,n):
        if not delta.count(i) == l:
            return False
    return True

In [89]:
print(difference_set([1,2,4],7)) #Example 1.2
print(difference_set([1,2,3],7)) #Example 1.2

True
False


We can generalise difference_set() function to direct product of cyclic groups

Direct product of two cyclic groups:

In [86]:
# subtraction_tuples defines subtraction of elements in Z_n x Z_m
def subtraction_tuples(tuple1, tuple2, n,m):
    x,y = tuple1
    z,w = tuple2
    return (x-z)%n,(y-w)%m


#tuples(n,m) returns a list of all elements in Z_n x Z_m
def tuples(n,m):
    list1 = []
    for i in range(n):
        for j in range(m):
            list1.append((i,j))
    return list1


#difference_set(l,n,m) returns True if elements in the list l form a difference set in Z_n x Z_m
def difference_set_tuples(l,n,m): #l is a list of tuples
    delta = [] #multiset of differences
    for i in l:
        for j in l:
            delta.append(subtraction_tuples(i,j,n,m)) 
    l = delta.count((1,0)) #number of times (1,0) appears in delta 
    ele = tuples(n,m) #all elements in the group Z_n X Z_m
    for i in ele[1:]: #check each non-identity elements appear same number of times
        if not delta.count(i) == l: 
            return False
    return True

In [87]:
difference_set_tuples([(0,0),(0,1),(0,2),(0,5),(1,0),(1,6)],2,8) #a difference set in Z_2 x Z_8

True

The above functions can be generalised to direct product of three cyclic groups by adding a parameter to each function. 

In [97]:
## subtraction_tuples_three defines subtraction of elements in Z_n x Z_m x Z_r
def subtraction_tuples_three(tuple1, tuple2, n,m,r):
    x,y,a = tuple1
    z,w,b = tuple2
    return (x-z)%n,(y-w)%m, (a-b)%r


#tuples_three(n,m,r) returns a list of all elements in Z_n x Z_m x Z_r
def tuples_three(n,m,r): 
    list1 = []
    for i in range(n):
        for j in range(m):
            for k in range(r):
                list1.append((i,j,k))
    return list1

##difference_set(l,n,m,r) returns True if elements in the list l form a difference set in Z_n x Z_m x Z_r
def difference_set_tuples_three(l,n,m,r): #l is a list of tuples
    delta = [] #multiset of differences
    for i in l:
        for j in l:
            delta.append(subtraction_tuples_three(i,j,n,m,r)) 
    l = delta.count((1,0,0)) #number of times (1,0) appears in delta 
    ele = tuples_three(n,m,r) #all elements in the group Z_n X Z_m
    for i in ele[1:]: #check each non-identity elements appear same number of times
        if not delta.count(i) == l: 
            return False
    return True

In [98]:
l = [(0,0,0),(1,0,0),(2,0,0),(0,1,0),(1,1,0),
(0,2,0),(0,0,1),(1,0,1),(0,1,1),(1,0,2),
(2,1,2),(0,2,2),(2,2,2)];
difference_set_tuples_three(l,3,3,3) #a difference set in Z_3 x Z_3 x Z_3

True

#### Counting equation
A $(v,k,\lambda)$-difference set satisfies the counting equation $k(k-1) = \lambda(v-1)$.

In [56]:
#basic_equation(v,k,l) returns True if (v,k,l) satisfites the formula above and False otherwise.
def basic_equation(v,k,l):
    if l*(v-1) == k*(k-1):
        return True
    else:
        return False

In [57]:
print(basic_equation(7,3,1))
print(basic_equation(7,4,1))

True
False


#### BRC theorem
A $(v,k,\lambda)$-difference set passes the BRC test:<br/>
* If $v$ is even, then $n = k - \lambda$ is a square. 
* If $v$ is odd, then the diophantine equation 
$x^2 = ny^2 + (-1)^\frac{v-1}{2}\lambda z^2$ has a non-trivial integer solution.

We first write some prelimary functions.

In [58]:
#primes(num) returns primes less than a postive integer num
def primes(num):
    q = []
    for j in range(1,num + 1):
        if j > 1:            
            for i in range(2, j):
                if (j % i) == 0:
                    break
            else:
                q.append(j)
    return(q)

#Square_free(n) returns True if an integer n is divisble by any power of prime and False otherwise
def Square_free(n):
    Prime_list = primes(int(n))
    i = 0
    while i < len(Prime_list):
        test =  Prime_list[i]*Prime_list[i]
        if test > abs(n):
            return True
        elif n%test == 0: 
            return False
        else:
            i = i+1
    return True

#quadratic_residue(n,m) returns True if n is a quardratic residue modulo m and False otherwise
def quardratic_residue(n,m):
    for i in range(0,m):
            if (i**2)%m != (n%m):
                continue
            else:
                return True
    return False           

Next we write the function for BRC theorem.

Note here we use the Lengendre's theorem to check if the diophantine equation has a nontrivial solution.

**Lengendre's theorem:**
Let $a,b \neq 0$ be integers with at least one of them positive(automatically true in our case). Let $d = \text{gcd}(a,b)$. Suppose each of $a,b$ are square free. Then the equation $x^2 = ay^2 + bz^2$ has a non-trivial integer solution $(x,y)$ if and only if:
* $a$ is a quadratic residue modulo $|b|$,
* $b$ is a quadratic residue modulo $|a|$, and 
* $-\frac{ab}{d^2}$ is a quadratic residue modulo $|d|$.


In [59]:
#BRC2(v,k,l) returns True if (v,k,l) passes the BRC test and False otherwise.
def BRC2(v,k,l): 
    if (int(v) % 2) == 0:   #If v is even
        n = int(k)-int(l)
        if math.sqrt(n).is_integer(): #we check if n = k-l is a perfect square
            return True
        else:
            return False   
        
#If v is odd        
    n = int(k-l)
    m = int((-1)**((v-1)/2)*int(l)) #diophantine equation: x^2 = ny^2 + mz^2

    if not (Square_free(n) and Square_free(m)): #If n or l is not square free, then
        return True      #the Lengendre's theorem does not apply thus we assume (v,k,l) passes the test                       
    else:          
        d = math.gcd(n,m)
        x = int(-n*m/d**2)
        if not quardratic_residue(n,abs(m)): #check n is a quardratic residue modulo |m|
             return False
        if not quardratic_residue(m,abs(n)): #check m is a quadratic residue modulo |n|
             return False
        if not quardratic_residue(x,abs(d)): #check -(nm/gcd(n,m)) is a quardratic residue modulo |gcd(n,m)|
             return False
        return True

In [60]:
print(BRC2(16,6,2)) #Example 2.3 in the report
print(BRC2(67,12,2)) #Example 2.5 in the report 

True
False


In [61]:
#This triple fails the BRC test but since 18 is not square-free, the Lengendre's theorem does not apply
#So we treat it passes BRC test
BRC2(93,24,6)

True

#### Find valid triples up to a certain upper bound for v
We want to find triples that pass the counting equation and BRC test up to a certain upper bound for v. <br/>
Note parameters for trivial difference sets and triples with $k>\frac{v}{2}$ are excluded.

In [62]:
#parameter_search3(v) return a list of such triples up to upper bound v
def parameter_search3(v):
    list1 = []
    for i in range(2,v+1):
        for j in range (int(sqrt(i-1)),i//2 +1): #Find triples with k<v/2. Note v-1 <= l(v-1)< k^2 so k > sqrt(i-1)
            for c in range(1,j//2 +1): #If k<v/2, then l<k/2
                if basic_equation(i,j,c) and BRC2(i,j,c): 
                    list1.append((i,j,c))
    return list1

In [63]:
a = parameter_search3(50)
print(a) 
print(len(a)) 
print(len(parameter_search3(300))) 

[(7, 3, 1), (11, 5, 2), (13, 4, 1), (15, 7, 3), (16, 6, 2), (19, 9, 4), (21, 5, 1), (23, 11, 5), (25, 9, 3), (27, 13, 6), (31, 6, 1), (31, 10, 3), (31, 15, 7), (35, 17, 8), (36, 15, 6), (37, 9, 2), (39, 19, 9), (40, 13, 4), (41, 16, 6), (43, 21, 10), (45, 12, 3), (47, 23, 11), (49, 16, 5)]
23
229


* 23 triples pass two tests with $v \leq 50$
* 229 triples pass two tests with $v \leq 300$

#### Equivalence
Two difference sets $D_1$ and $D_2$ are **equivalent** if $D_2 = g\alpha(D_1)$ for some $g \in G$ and $\alpha \in \mathrm{Aut}(G)$. <br/>
Equivalent difference sets are considered the same in the classification of all difference sets in a group.

In [64]:
#coprime_list(num) returns a list of numbers less than and coprime with num
def coprime_list(num):
    t = []
    for i in range(0,num):
        if math.gcd(i,num) == 1:
            t.append(i)
    return t


#equivalence(D1,D2,n) returns True if D1 and D2 are equivalent difference sets in the CYCLIC group of order n. 
#D1 and D2 are lists and n is an integer.
def equivalence(D1,D2,n):
    automorphism = coprime_list(n)  #an automorphism in Z_n can be specifed as a number coprime with n
    
    #First transform D1 and D2 to difference sets(lists) containing 0(i.e., identity)
    if not 0 in D1:                
        D1 = [(x - D1[0])%n for x in D1]        
    if not 0 in D2:
        D2 = [(x-D2[0])%n for x in D2]
               
    for alpha_i in automorphism:
        image_D1 = [alpha_i*x %n for x in D1]
        for j in D2:  #g must be an element in D2 since 0 in the alpha_i(D1)
            newlist = [(x+j) %n for x in image_D1]
            if set(D2) == set(newlist):
                return True
    return False

In [65]:
equivalence([1,2,4],[3,5,6],7)

True

Note here we only implement the algorithm for cyclic difference sets since it is much easier to find an automorphisms in a cyclic group

## 2. Multiplier Theorems.

#### First Multiplier Theorem
Let $D$ be a $(v,k,\lambda)$-difference set in an abelian group $G$. Then a prime $p$ is a numerical multiplier if 
$p>\lambda$, $p\mid n$, and $p\nmid v $.


In [66]:
#MultiplierTheorem1(v,k,l) returns a list of multipliers
#for all abelian (v,k,l) difference sets using first multiplier theorem
def MultiplierTheorem1(v,k,l):
    p = []
    n = k-l
    q = primes(n) #Find primes less than n
    for i in q:
        if (i > l) and (n % i == 0) and (v % i != 0):
                p.append(i)
    return p

In [67]:
MultiplierTheorem1(27,13,6)

[7]

#### Second Multiplier Theorem
Let $G$ be an abelian group with exponent $v^\ast$ and $D$ be a $(v,k,\lambda)$-difference set. 
Let $m$ be a divisor of $n$ with $m>\lambda$ and $\text{gcd}(m,v)=1$. Then an integer $t$ with $\mathrm{gcd}(t,v)=1$ is a numerical multiplier if, for all prime $p$ such that $p\mid m$, $\exists f\in \mathbb{Z}_{\geq 0}$ such that $t \equiv p^f\mod v^\ast$.

In [68]:
#primes_factors2(n) returns the prime factors of an integer n
#reference: https://stackoverflow.com/questions/16996217/prime-factorization-list
def primes_factors2(n):
    primfac = []
    while n%2 == 0:
        primfac.append(2)
        n = n//2
    d = 3
    while d*d <= n:
        while (n % d) == 0:
            primfac.append(d)  # supposing you want multiple factors repeated
            n //= d
        d += 2
    if n > 1:
        primfac.append(n)
    return list(set(primfac))


#MultiplierTheorem2(v,k,l,v_e) returns a list of multipliers using second multiplier theorem
#for all abelian (v,k,l) difference sets in groups of order v with exponent v
def MultiplierTheorem2(v,k,l,v_e):
    r = []
    n = k-l
    for m in range(l+1, n+1):  #let m be a divisor of n, coprime with v, and m > l
        if (n%m == 0) and (math.gcd(m,v)==1): 
            q_list = primes_factors2(m) #a list of primes that divides m
            t_list = coprime_list(v)   #a list of integers coprime with v.
            for t in t_list:
                count = 0  #count p in the q_list satisfies the condition(there exists f s.t t = p^f mod v_e)  
                while count < len(q_list):
                    count1 = count  
                    p = q_list[count] 
                    for f in range(1,v):
                        if t%v_e != p**f %v_e:  #if there exists p such that the condition does not hold
                            count = len(q_list) + 1  #set count = len(q_list) and test other p 
                        else:
                            count = count1 + 1 #if there exists f such that t = p^f mod v_e, set count += 1
                            break   # then break the loop(Note count may already be set to len(q_list+1))
                                    #so we introduce count1
                if count == len(q_list):  #if all p satisfies the condition, then t is a numerical multiplier
                    r.append(t)
    return r

In [69]:
print(MultiplierTheorem2(27,13,6,27))  #Z_27
print(MultiplierTheorem2(27,13,6,9))   #Z_9 X Z_3
print(MultiplierTheorem2(27,13,6,3))   #Z_3 X Z_3 X Z_3

[1, 4, 7, 10, 13, 16, 19, 22, 25]
[1, 4, 7, 10, 13, 16, 19, 22, 25]
[1, 4, 7, 10, 13, 16, 19, 22, 25]


#### Comparison
* The first multiplier theorem finds a multiplier but the second multipier theorem finds a group of multipliers 
* The first multiplier theorem does not require the exponent of the groups
* The second multiplier theorem is a generalisation of the first multiplier theorem as an attempt to prove the multiplier conjecture

In [70]:
print(MultiplierTheorem1(35,17,8))
print(MultiplierTheorem2(35,17,8,35))

[]
[1, 3, 4, 9, 11, 12, 13, 16, 17, 27, 29, 33]


#### Aside: The most general Multiplier Theorem.
Currently the most general multiplier theorem is Theorem 3.1 in the paper 'A survey of the multiplier conjecture' of Daniel M. Gordon & Bernhard Schmidt (link: https://link.springer.com/article/10.1007/s10623-015-0153-8). <br/>
We also implement this theorem. Unfortunately, it takes much slower to run compared with the 1st and 2nd multiplier theorems, so we did not use it in our project. 

In [71]:
#divisor(n) returns all divisors of a number n
def divisor(n):
    l = []
    for i in range(1,n+1):
        if n%i == 0:
            l.append(i)
    return l

#Given t, u, v and check if there exists a positive integer f such that t = u^f(mod v).
def rasing_power(t,u,v):
    lcm = u*v//math.gcd(u,v)
    order = lcm//u #order of u
    for i in range(1,order+1):
        if not t%v == u**i %v:
            continue
        else:
            return True
    return False

#check that for a list
def rasing_power_list(t,l,v):
    for i in l:
        if not rasing_power(t,i,v):
            return False
    return True

#self_modulo(n,v) returns True if a prime p is a self-conjugate_modulo v and False otherwise
def self_conjugate_modulo(p,v):
    a = 0
    v1 = v
    while v1%p == 0:
        a += 1
        v1 = v1//p
    w = v//(p**a)
  #  lcm =  p*v//math.gcd(v,p)
  #  order = lcm//p
    if rasing_power(-1,p,w):
         return True
    else:
        return False


#Check is a list of primes are self-conjugate_modolo v
def self_conjugate_modulo_list(l,v):
    for i in l:
        if not self_conjugate_modulo(i,v):
            return False
    return True

#M function used in Theorem 3.1 on Godron and Schmidt paper
def M(m,b):
    if m == 1:
        return 1
    else:
        p_list = primes_factors2(m)
        p = p_list[0] #take a prime divisor of m 
        e = 0 #count the highest power p^e dividing m
        m1 = m
        while m1%p == 0:
            e += 1
            m1 = m1//p
        factors = p_list + primes_factors2(M(m**2//p**(2*e),2*m**2//p**(2*e)-2))
        for i in range(1,b+1):
            factors = factors + primes_factors2(p**i-1)   
        return math.prod(set(factors))

#The most general multiplier theorem(Theorem 3.1 in the paper of Gordan and Schimidt)
#This function is slower than MultiplierTheorem2
def MultiplierTheorem3(v,k,l,v_e):
    multipliers = []
    n = k-l
    n1_list = divisor(n)
    t_list = coprime_list(v)
    for n1 in n1_list:
        u_list = primes_factors2(n1)
        for t in t_list:
            if not (self_conjugate_modulo_list(u_list,v_e) or rasing_power_list(t,u_list,v_e)):
                continue
            else:
                if n1/math.gcd(v,n1) > l or math.gcd(v,M(n*math.gcd(v,n1)//n1, k*math.gcd(v,n1)//n1)) == 1:
                    multipliers.append(t)
    return list(set(multipliers))

In [75]:
MultiplierTheorem3(27,13,6,27)

[1, 4, 7, 10, 13, 16, 19, 22, 25]

In [72]:
start = time.time()
MultiplierTheorem1(27,13,6)
end = time.time()
end - start

0.00014781951904296875

In [73]:
start = time.time()
MultiplierTheorem2(27,13,6,27)
end = time.time()
end - start

0.0006539821624755859

In [74]:
start = time.time()
MultiplierTheorem3(27,13,6,27)
end = time.time()
end - start

0.25290393829345703

Computation cost: Theorem 3.1 > second multiplier theorem > first multiplier theorem

## 3. Find difference sets using multipliers.

#### Algorithm for classifying difference sets for a valid parameter using multipliers. 

* Find a left multiplier (here we use the second multiplier theorem).
* Calculate the orbits under the action of the group generated by the multiplier.
* Find a union of orbits that has $k$ elements in total and test if it is a difference set. 
* Check if any of these difference sets are equivalent

##### cyclic difference sets:

In [77]:
#orbit(x,n,t) returns a list containing elements of the orbits of x under the action of <phi_t> in the group Z_n
def orbit(x,n,t):
    list_1 = [x]
    while True:
        if t*list_1[-1] % n == x:
            return list_1
        else:
            list_1.append(t*list_1[-1] % n)

#orbit_cyclic(n,t) returns a list, which first entry is a list containing
#the orbits of Z_n under the action of <phi_t>. The second entry is a list containing the size of each orbit
def orbit_cyclic(n,t):
    list1 = list(range(n))
    lists = [] #lists will contain the orbits
    sizes = []
    while len(list1):
            i = list1[0]
            list2 = orbit(i,n,t)
            lists.append(list2)
            sizes.append(len(list2))
            list1 = [x for x in list1 if x not in list2]
            if len(list1) == 0:
                break
    return lists,sizes

In [79]:
orbit_cyclic(7,2) #orbits of Z_7 under the numerical multiplier 2

([[0], [1, 2, 4], [3, 6, 5]], [1, 3, 3])

In [94]:
#find (v,k,l)-difference sets up to equivalence in Z_v using the second multiplier theorem
def find_difference_set_explictly(v,k,l):
    list_of_difference_sets = []
    list1 = MultiplierTheorem2(v,k,l,v)
    if not list1:
        print("No mutipliers")
        return 
    t = list1[1] #skip multiplier 1
    list2 = orbit_cyclic(v,t)
    dic = {} #create a dictionary that contains orbits as the keys and size of orbits as value
    for i in range(0,len(list2[0])):
        dic[tuple(list2[0][i])] = list2[1][i]
        for key,value in dict(dic).items(): #remove cycles greater than k. 
            if value > k:    #reference: https://stackoverflow.com/questions/29218750/what-is-the-best-way
                del dic[key]  #-to-remove-a-dictionary-item-by-value-in-python
    for z in range(1,len(dic)+1):
        combinations = itertools.combinations(dic,z)  #choose i keys from dic
        for j in combinations:  #check each combination of z elements(combination denoted j) from a difference set
            s = 0         #calculate the total length of these z elements
            for d in j:
                s += dic[d]
            if s == k:  #if the length is k, then we check if it is a difference set
                l = []
                for ch in j:
                    l = l + list(ch) #transform j to a list l
                if difference_set(l,v): #if l is a difference set
                    if not list_of_difference_sets:  #add l if the list of difference sets is empty
                        list_of_difference_sets.append(l)
                        continue
                    if list_of_difference_sets: #if we already have difference sets
                        count = 0
                        for diff in list_of_difference_sets: #we need to check l is inequivalent to any of them 
                            if equivalence(l,diff,v):
                                count = 1
                                break
                        if count == 0:
                            list_of_difference_sets.append(l)
    print(list_of_difference_sets)
    return

In [95]:
find_difference_set_explictly(7,3,1)

[[1, 2, 4]]


##### difference sets in the direct product of two cyclic groups:

In [104]:
#define multiplication of tuple by a scalar in Z_n x Z_m
def multiply_tuple(t,mytuple,n,m):
    x,y = mytuple
    return t*x%n, t*y%m


#orbit(mytuple,n,m,t) returns the orbit of an element mytuple in Z_n x Z_m
def orbit_tuple(mytuple,n,m,t): #t must be a power automorphism for Z_n x Z_m. Otherwise the function won't stop!
    list_1 = [mytuple]
    while True:
        y = multiply_tuple(t,list_1[-1],n,m)
        if y == mytuple:
            return list_1
        else:
            list_1.append(y)


#orbit_direct_sum(n,m,t) computes the orbits of Z_n x Z_m under the action of <phi_t>
#It returns a list whose first element is a list of orbits and the second element is a list of lengths of the orbits
def orbit_direct_sum(n,m,t):
    list1 = tuples(n,m)
    lists = [] #lists will contain the orbits
    sizes = []
    while len(list1):
            i = list1[0]
            list2 = orbit_tuple(i,n,m,t)
            lists.append(list2)
            sizes.append(len(list2))
            list1 = [x for x in list1 if x not in list2]
            if len(list1) == 0:
                break
    return lists,sizes

In [103]:
orbit_direct_sum(5,5,2) #orbits of Z_5 x Z_5 using numerical multiplier 2

([[(0, 0)],
  [(0, 1), (0, 2), (0, 4), (0, 3)],
  [(1, 0), (2, 0), (4, 0), (3, 0)],
  [(1, 1), (2, 2), (4, 4), (3, 3)],
  [(1, 2), (2, 4), (4, 3), (3, 1)],
  [(1, 3), (2, 1), (4, 2), (3, 4)],
  [(1, 4), (2, 3), (4, 1), (3, 2)]],
 [1, 4, 4, 4, 4, 4, 4])

We then write a function to find the difference sets in direct product of $\mathbb{Z}_n \times \mathbb{Z}_m$ given a multiplier $t$. <br/>
Note as we did not implement an algortithm to find the automorphism groups in a non cyclic group, we cannot check euquivalence of difference sets in this case.

In [106]:
#Given a paramter (v,k,l), the multiplier t, find the difference sets in Z_n x Z_m
#If no difference sets exist, the function print nothing
def find_difference_set_explictly_tuples(v,k,l,n,m,t):
    list2 = orbit_direct_sum(n,m,t) #orbits of Z_n x Z_m under t
    dic = {} #create a dictionary that contains orbits as the keys and size of orbits as value
    for i in range(0,len(list2[0])):
        dic[tuple(list2[0][i])] = list2[1][i]
        for key,value in dict(dic).items(): #remove cycles greater than k. 
            if value > k:    #reference: https://stackoverflow.com/questions/29218750/what-is-the-best-way
                del dic[key]  #-to-remove-a-dictionary-item-by-value-in-python
    for z in range(1,len(dic)+1):
        combinations = itertools.combinations(dic,z)  #choose i keys from dic
        for j in combinations:  #check each combination of z elements(combination denoted j) from a difference set
            s = 0         #calculate the total length of these z elements
            for d in j:
                s += dic[d]
            if s == k:  #if the length is k, then we check if it is a difference set
                l = []
                for ch in j:
                    l = l + list(ch) #transform j to a list l
                if difference_set_tuples(l,n,m): #if l is a difference set
                    print(l)

In [108]:
find_difference_set_explictly_tuples(25,9,3,5,5,2) #prints nothing--no (29,9,3)-difference sets exist in Z_5 x Z_5. 

In [116]:
find_difference_set_explictly_tuples(63,31,13,21,3,2) #We don't check equivalence here

[(0, 0), (1, 0), (2, 0), (4, 0), (8, 0), (16, 0), (11, 0), (1, 1), (2, 2), (4, 1), (8, 2), (16, 1), (11, 2), (3, 0), (6, 0), (12, 0), (3, 1), (6, 2), (12, 1), (3, 2), (6, 1), (12, 2), (5, 1), (10, 2), (20, 1), (19, 2), (17, 1), (13, 2), (9, 0), (18, 0), (15, 0)]
[(0, 0), (1, 0), (2, 0), (4, 0), (8, 0), (16, 0), (11, 0), (1, 2), (2, 1), (4, 2), (8, 1), (16, 2), (11, 1), (3, 0), (6, 0), (12, 0), (3, 1), (6, 2), (12, 1), (3, 2), (6, 1), (12, 2), (5, 2), (10, 1), (20, 2), (19, 1), (17, 2), (13, 1), (9, 0), (18, 0), (15, 0)]
[(0, 0), (1, 0), (2, 0), (4, 0), (8, 0), (16, 0), (11, 0), (3, 0), (6, 0), (12, 0), (5, 1), (10, 2), (20, 1), (19, 2), (17, 1), (13, 2), (5, 2), (10, 1), (20, 2), (19, 1), (17, 2), (13, 1), (9, 0), (18, 0), (15, 0), (9, 1), (18, 2), (15, 1), (9, 2), (18, 1), (15, 2)]
[(0, 0), (1, 1), (2, 2), (4, 1), (8, 2), (16, 1), (11, 2), (1, 2), (2, 1), (4, 2), (8, 1), (16, 2), (11, 1), (3, 0), (6, 0), (12, 0), (3, 1), (6, 2), (12, 1), (3, 2), (6, 1), (12, 2), (5, 0), (10, 0), (20, 

##### difference sets in the direct product of three cyclic groups are similar:

In [110]:
#Slightly change the code(i.e., the length of the tuple)

#define multiplication of tuple by a scalar in Z_n x Z_m x Z_r
def multiply_tuple_three(t,mytuple,n,m,r):
    x,y,z = mytuple
    return t*x%n, t*y%m, t*z%r

#orbit(mytuple,n,m,r,t) returns the orbit of an element mytuple in Z_n x Z_m x Z_r under the action of <phi_t>
def orbit_tuple_three(mytuple,n,m,r,t): #t must be a power automorphism for Z_n x Z_m x Z_r.
    list_1 = [mytuple]
    while True:
        y = multiply_tuple_three(t,list_1[-1],n,m,r)
        if y == mytuple:
            return list_1
        else:
            list_1.append(y)
            

#orbit_direct_sum(n,m,r,t) computes the orbits of Z_n x Z_m x Z_r given a multiplier t
def orbit_direct_sum_three(n,m,r,t):
    list1 = tuples_three(n,m,r)
    lists = [] #lists will contain the orbits
    sizes = []
    while len(list1):
            i = list1[0]
            list2 = orbit_tuple_three(i,n,m,r,t)
            lists.append(list2)
            sizes.append(len(list2))
            list1 = [x for x in list1 if x not in list2]
            if len(list1) == 0:
                break
    return lists,sizes

#Given a paramter (v,k,l), the multiplier t, find the difference sets in Z_n x Z_m x Z_r
#If no difference sets exist, the function print nothing
def find_difference_set_explictly_tuples_three(v,k,l,n,m,r,t):
    list2 = orbit_direct_sum_three(n,m,r,t) #orbits of Z_n X Z_m under t
    dic = {} #create a dictionary that contains orbits as the keys and size of orbits as value
    for i in range(0,len(list2[0])):
        dic[tuple(list2[0][i])] = list2[1][i]
        for key,value in dict(dic).items(): #remove cycles greater than k. 
            if value > k:    #reference: https://stackoverflow.com/questions/29218750/what-is-the-best-way
                del dic[key]  #-to-remove-a-dictionary-item-by-value-in-python
    for z in range(1,len(dic)+1):
        combinations = itertools.combinations(dic,z)  #choose i keys from dic
        for j in combinations:  #check each combination of z elements(combination denoted j) from a difference set
            s = 0         #calculate the total length of these z elements
            for d in j:
                s += dic[d]
            if s == k:  #if the length is k, then we check if it is a difference set
                l = []
                for ch in j:
                    l = l + list(ch) #transform j to a list l
                if difference_set_tuples_three(l,n,m,r): #if l is a difference set
                    print(l)

In [112]:
find_difference_set_explictly_tuples_three(40,13,4,10,2,2,3)#no (40,13,4)-difference set exists in Z_10 x Z_2 x Z_2

## 4. Difference sets in $D_{2n}$  where $n\leq 150$

It has been long conjectured that no difference set exists in dihedral groups $D_{2n}$. <br/>
We want to use multipliers to study this conjecture. However, multipliers theorems only work for abelia groups. <br/>
So we introduce Dillon's dihedral trick, which builds a bridge between cyclic difference sets and difference sets in $D_{2n}$.

##### Dillon's dihedral trick
Suppose $D_{2n}$ contains a $(v,k,\lambda)$-difference set, then $\mathbb{Z}_{2n}$ contains a $(v,k,\lambda)$-difference set.

So we only need to use multipliers to show no difference sets exist in $\mathbb{Z}_{2n}$. <br/>
However, there are some paramters where multiplier theorems do not work. In particular, difference sets in Hadamard family (i.e., v = 4n = 4(k-$\lambda$)). For example, (16,6,2) since 6-2$\mid$16). <br/>
We introduce Turyn's exponent bound to deal with such cases.

##### Turyn's exponant bound
No $(4p^{2a},2p^{2a}-p^a, p^{2a}-p^a)$-difference sets exist in a cyclic group of order $4p^{2a}$ where $p$ is a prime and $a\geq 1$ is an integer. 

In [118]:
#rule_out_case_4pt(v,k,l) returns True if (v,k,l) does not have a form in Turyn's and 
#returns False if (v,k,l) is ruled out in our study by Turyn's.
def rule_out_case_4pt(v,k,l):
    if v%4 != 0:
        return True
    p1 = int(v/4)
    primes = primes_factors2(p1)
    if len(primes) == 1: #check if p1 is a power of p. 
        p = primes[0]
        a = 1
        while p**2 < p1:  #check if p1 is a even power of p
            p1 = p1/p**2
            a += 1
        if p1 == p: 
            return True
        else:
            if (k == 2*p**(2*a) - p**a) and (l == p**(2*a) - p**a):  #check k and l
                return False   #rules out by Turyn's exponent bound
            else:
                return True
    else:
        return True

In [119]:
rule_out_case_4pt(16,6,2)

False

##### Difference sets in dihedral groups.

In [125]:
#remove_item_greater_than(l,n) removes items in a list l that is greater than n
def remove_item_greater_than(l,n):
    l2 = [x for x in l if x<=n]
    l2.sort()
    return l2

#using second multiplier theorem to check the existence of (v,k,l)-difference set in Z_n
def find_difference_set(v,k,l):
    list1 = MultiplierTheorem2(v,k,l,v)
    if not list1: 
        return("No multipliers for",v,k,l)
    t = list1[1] #skip multiplier 1
    size = orbit_cyclic(v,t)[-1] #compute the length of each cyclc
    size2 = remove_item_greater_than(size,k) #remove cycles greater than k to save computation time. Note size2 is sroted
    for i in range(1,len(size2)+1):
        if sum(size2[0:i+1]) > k:  #if the sum of i smallest cycles greater than k
            continue                #then we don't need to check any other combinations of i cycles 
        combinations = itertools.combinations(size2,i)  
        for j in combinations:
            if sum(j) == k:
                 return(v,k,l," possibly have a difference set")
    return(v,k,l," does not contain difference set")


#check status of the existence of difference sets to an upper bound v
def dihedral_check(v):
    list1 = parameter_search3(v)
    for ch in list1:
        v1,k,l = ch
        if v1%2 != 0: #only check for even v
            continue           
        if not rule_out_case_4pt(v1,k,l): #check if ruled out by Turyn's
            print(v1,k,l," rule out because Turyn's exponent bound")
            continue
        result = find_difference_set(v1,k,l)
        print(result)

In [126]:
dihedral_check(300)

16 6 2  rule out because Turyn's exponent bound
36 15 6  rule out because Turyn's exponent bound
(40, 13, 4, ' possibly have a difference set')
(56, 11, 2, ' possibly have a difference set')
64 28 12  rule out because Turyn's exponent bound
('No multipliers for', 66, 26, 10)
('No multipliers for', 70, 24, 8)
('No multipliers for', 78, 22, 6)
('No multipliers for', 96, 20, 4)
100 45 20  rule out because Turyn's exponent bound
(112, 37, 12, ' possibly have a difference set')
('No multipliers for', 120, 35, 10)
('No multipliers for', 144, 66, 30)
('No multipliers for', 154, 18, 2)
(156, 31, 6, ' possibly have a difference set')
('No multipliers for', 160, 54, 18)
('No multipliers for', 176, 50, 14)
196 91 42  rule out because Turyn's exponent bound
(204, 29, 4, ' does not contain difference set')
('No multipliers for', 208, 46, 10)
('No multipliers for', 210, 77, 28)
(220, 73, 24, ' possibly have a difference set')
256 120 56  rule out because Turyn's exponent bound
('No multipliers for',

## 5. Classification of abelian difference sets with $v\leq 50$.

In [134]:
start = time.time() #start timer

First check cyclic cases

In [135]:
list1 = parameter_search3(50) #list of valid parameters 
for ch in list1:
    v,k,l = ch
    print("The difference sets of parameter ",v,k,l," are")
    find_difference_set_explictly(v,k,l)  

The difference sets of parameter  7 3 1  are
[[1, 2, 4]]
The difference sets of parameter  11 5 2  are
[[1, 3, 9, 5, 4]]
The difference sets of parameter  13 4 1  are
[[0, 1, 3, 9]]
The difference sets of parameter  15 7 3  are
[[0, 1, 2, 4, 8, 5, 10]]
The difference sets of parameter  16 6 2  are
No mutipliers
The difference sets of parameter  19 9 4  are
[[1, 4, 16, 7, 9, 17, 11, 6, 5]]
The difference sets of parameter  21 5 1  are
[[3, 6, 12, 7, 14]]
The difference sets of parameter  23 11 5  are
[[1, 2, 4, 8, 16, 9, 18, 13, 3, 6, 12]]
The difference sets of parameter  25 9 3  are
[]
The difference sets of parameter  27 13 6  are
[]
The difference sets of parameter  31 6 1  are
[[1, 5, 25, 11, 24, 27]]
The difference sets of parameter  31 10 3  are
[]
The difference sets of parameter  31 15 7  are
[[1, 2, 4, 8, 16, 3, 6, 12, 24, 17, 15, 30, 29, 27, 23], [1, 2, 4, 8, 16, 5, 10, 20, 9, 18, 7, 14, 28, 25, 19]]
The difference sets of parameter  35 17 8  are
[[0, 1, 3, 9, 27, 11, 33, 29,

Next check those cases can be written as direct product of cyclic groups.<br/>
Note multiplier theorems does not apply to (16,6,2),(36,15,6) and (45,12,3) since n$\mid$v

In [136]:
find_difference_set_explictly_tuples(25,9,3,5,5,2) #(25,9,3) in Z_5 x Z_5

In [137]:
find_difference_set_explictly_tuples(27,13,6,9,3,7) #(27,13,6) in Z_9 x Z_3

In [138]:
find_difference_set_explictly_tuples(40,13,4,20,2,3) #(40,13,4) in Z_20 x Z_2

In [139]:
find_difference_set_explictly_tuples(49,16,5,7,7,11) #(49,16,5) in Z_7 x Z_7

In [140]:
find_difference_set_explictly_tuples_three(40,13,4,10,2,2,3)

In [141]:
#find_difference_set_explictly_tuples_three(27,13,6,3,7)

In [142]:
end = time.time()
end - start

1.9276750087738037

It is important to note, a straightforward running function may get into trouble. For example, (27,13,6) in $\mathbb{Z}_3 \times \mathbb{Z}_3 \times \mathbb{Z}_3$ has a multiplier 7 but its orbits all have length 1. The computation is quite large compared with other cases.

In [143]:
MultiplierTheorem2(27,13,6,3)

[1, 4, 7, 10, 13, 16, 19, 22, 25]

In [145]:
orbit_direct_sum_three(3,3,3,7)

([[(0, 0, 0)],
  [(0, 0, 1)],
  [(0, 0, 2)],
  [(0, 1, 0)],
  [(0, 1, 1)],
  [(0, 1, 2)],
  [(0, 2, 0)],
  [(0, 2, 1)],
  [(0, 2, 2)],
  [(1, 0, 0)],
  [(1, 0, 1)],
  [(1, 0, 2)],
  [(1, 1, 0)],
  [(1, 1, 1)],
  [(1, 1, 2)],
  [(1, 2, 0)],
  [(1, 2, 1)],
  [(1, 2, 2)],
  [(2, 0, 0)],
  [(2, 0, 1)],
  [(2, 0, 2)],
  [(2, 1, 0)],
  [(2, 1, 1)],
  [(2, 1, 2)],
  [(2, 2, 0)],
  [(2, 2, 1)],
  [(2, 2, 2)]],
 [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1])

Other multipliers give the same result since they are all in the group generated by 7. (Recall multipliers is a subgroup of automophism group)

Except for some exceptional cases, our program can work for difference sets with $v<500$.

In [148]:
start = time.time()
find_difference_set_explictly(499,249,124)
end = time.time()

[[1, 4, 16, 64, 256, 26, 104, 416, 167, 169, 177, 209, 337, 350, 402, 111, 444, 279, 118, 472, 391, 67, 268, 74, 296, 186, 245, 481, 427, 211, 345, 382, 31, 124, 496, 487, 451, 307, 230, 421, 187, 249, 497, 491, 467, 371, 486, 447, 291, 166, 165, 161, 145, 81, 324, 298, 194, 277, 110, 440, 263, 54, 216, 365, 462, 351, 406, 127, 9, 36, 144, 77, 308, 234, 437, 251, 6, 24, 96, 384, 39, 156, 125, 5, 20, 80, 320, 282, 130, 21, 84, 336, 346, 386, 47, 188, 253, 14, 56, 224, 397, 91, 364, 458, 335, 342, 370, 482, 431, 227, 409, 139, 57, 228, 413, 155, 121, 484, 439, 259, 38, 152, 109, 436, 247, 489, 459, 339, 358, 434, 239, 457, 331, 326, 306, 226, 405, 123, 492, 471, 387, 51, 204, 317, 270, 82, 328, 314, 258, 34, 136, 45, 180, 221, 385, 43, 172, 189, 257, 30, 120, 480, 423, 195, 281, 126, 22, 88, 352, 410, 143, 73, 292, 170, 181, 225, 401, 107, 428, 215, 361, 446, 287, 150, 101, 404, 119, 476, 407, 131, 25, 100, 400, 103, 412, 151, 105, 420, 183, 233, 433, 235, 441, 267, 70, 280, 122, 488, 45

In [149]:
end-start

1.2998912334442139

## 6. Contribution in coding part

**Samuel Hudson** wrote the code for checking the basic equations to see what group orders could possibly contain difference sets, the code for the Bruck Ryser Chowla theorem, and the code for the first and second multiplier theorems. 

**Lijun Zou** involved in revising the code for BRC theorem and the second multiplier theorem, wrote other parts which are not coverd above, and tidied up the code.